# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/Qwen/Qwen-Image-Edit',
 '/deepseek-ai/DeepSeek-V3.1-Base',
 '/deepseek-ai/DeepSeek-V3.1',
 '/google/gemma-3-270m',
 '/ByteDance-Seed/Seed-OSS-36B-Instruct',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Qwen/Qwen-Image-Edit',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/Qwen/Qwen-Image',
 '/spaces/webml-community/dinov3-web',
 '/spaces',
 '/datasets/nvidia/Granary',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/nvidia/Llama-Nemotron-VLM-Dataset-v1',
 '/datasets/allenai/WildChat-4.8M',
 '/datasets/nvidia/Nemotron-CC-v2',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transforme

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'forum', 'url': 'https://discuss.huggingface.co'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Qwen/Qwen-Image-Edit
Updated
4 days ago
•
23.5k
•
1.05k
deepseek-ai/DeepSeek-V3.1-Base
Updated
about 9 hours ago
•
10.9k
•
847
deepseek-ai/DeepSeek-V3.1
Updated
about 9 hours ago
•
7.8k
•
422
google/gemma-3-270m
Updated
8 days ago
•
57.7k
•


In [21]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nQwen/Qwen-Image-Edit\nUpdated\n4 days ago\n•\n23.5k\n•\n1.05k\ndeepseek-ai/DeepSeek-V3.1-Base\nUpdated\nabout 9 hours ago\n•\n10.9k\n•\n847\ndeepseek-ai/DeepSeek-V3.1\nUpdated\nabout 9 hours ago\n•\n7.8k\n•\n422\ngoogle/gemma-3-270m\nUpdated\n8 days ago\n•\n57.7k\n•\n589\nByteDance-Seed/Seed-OSS-36B-Instruct\nUpdated\nabout 9 hours ago\n•\n2.15k\n•\n213\nBrowse 1M+ models\nSpaces\nRunning\

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}


# Hugging Face Brochure

## Company Overview
**Hugging Face** is at the forefront of building an AI-centric community dedicated to shaping the future of machine learning. Our platform is a collaborative space for developers, researchers, and enterprises to explore and share a vast array of resources, including over **1 million models**, **250,000 datasets**, and innovative applications across various modalities like text, images, video, audio, and 3D. 

## Our Vision
_Hugging Face_ is committed to creating tools and resources that empower individuals and organizations to engage with artificial intelligence seamlessly. We believe in fostering an open-source culture that accelerates machine learning development and democratizes access to advanced AI technologies.

## Community & Collaboration
Our platform thrives on community contributions, encouraging collaboration among developers and researchers. We currently host **more than 50,000 organizations**, including tech giants such as Google, Amazon, Microsoft, and Meta. This extensive network facilitates the rapid sharing of knowledge and resources, making it easier for anyone to get involved in machine learning.

## Our Products
- **Models**: With **1M+ models**, users can explore trending AI models and leverage them in their projects.
- **Datasets**: Access a wide range of datasets suitable for any machine learning tasks, ensuring scalability and efficiency.
- **Spaces**: Experiment with and deploy applications with minimal setup through optimized environments, allowing users to create innovative ML applications.

## Enterprise Solutions
Hugging Face also offers **Enterprise solutions** tailored for teams requiring advanced tools with added security and dedicated support. Our paid Compute options allow organizations to manage resources effectively while benefiting from enterprise-grade access controls and collaborative tools.

### Pricing
- **Compute Solutions**: Starting at **$0.60/hour** for GPU usage.
- **Team & Enterprise Packages**: Starting at **$20/user/month**—providing enhanced features including single sign-on, priority support, and more.

## Company Culture
At Hugging Face, we value **innovation**, **collaboration**, and **community engagement**. We promote a culture where every team member can share ideas and contribute to our mission of transforming the landscape of AI. Our open-source roots inspire transparency and collective growth, allowing employees to thrive in a supportive environment.

## Careers at Hugging Face
We are always on the lookout for passionate individuals eager to dive into the world of AI and ML. Working at Hugging Face not only means you become part of a dynamic team, but it also allows you to influence the future of technology. Check our [careers page](https://huggingface.co/jobs) for available opportunities.

## Join Us
Whether you are a developer, a data scientist, or an AI enthusiast, Hugging Face welcomes you to explore our offerings and contribute to the collaborative AI landscape. → [Explore More](https://huggingface.co)

---
For more detailed information, visit our [website](https://huggingface.co) or reach out via our community channels on [GitHub](https://github.com/huggingface), [Twitter](https://twitter.com/huggingface), [LinkedIn](https://www.linkedin.com/company/huggingface), and [Discord](https://discord.gg/huggingface).

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face Brochure

---

## Welcome to Hugging Face
**The AI Community Building the Future**

At Hugging Face, we believe in the power of collaboration to drive innovation in Artificial Intelligence. Our platform empowers the machine learning community to explore, create, and share models, datasets, and applications, all in one place. 

---

## What We Offer

### **Explore & Collaborate**
- **Models**: Browse over **1 million models** including state-of-the-art options like Qwen, DeepSeek, and many more.
- **Datasets**: Access an extensive library of **250,000+ datasets** to accelerate your machine learning projects.
- **Spaces**: Launch applications easily with our user-friendly tools that support text, images, video, audio, and 3D content.

### **Enterprise Solutions**
With our enterprise-grade services, teams can leverage optimized solutions for AI development, ensuring security and dedicated support as they scale their operations.

---

## Our Customers

Join **50,000+ organizations** transforming their AI strategies with Hugging Face, including industry leaders like:
- **Google**
- **Microsoft**
- **Amazon**
- **Meta**

From non-profits to Fortune 500 companies, we serve a diverse clientele composed of innovators in AI and machine learning.

---

## Join Our Community

Hugging Face is more than just a platform; it's a thriving community of over **55,000 AI & ML enthusiasts**. We invite you to participate in discussions, collaborate on cutting-edge research, and contribute to an inclusive environment where everyone can share their knowledge and creativity.

---

## Careers at Hugging Face

At Hugging Face, we value every team member's contribution. We foster a culture of **growth, innovation, and support**. We're always on the lookout for passionate individuals who are eager to build meaningful AI solutions. Explore our open positions and become part of our mission to democratize AI!

### **Current Openings**
- Machine Learning Engineers
- Data Scientists
- Product Managers
- Community Managers

---

## Get Started

Visit us at [Hugging Face](https://huggingface.co) to learn more about our offerings, explore the community, or sign up today! Join us in building the future of AI together.

--- 

### **Contact Us**
For inquiries related to partnerships or enterprise solutions, feel free to reach out via our website or connect with us through social media.

---

*Together, let's harness the power of AI for a better tomorrow!*

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# HuggingFace: Not Your Average AI Company

## Welcome to the Hugging Face Family!

Where not only the AI community builds the future, but we also throw in a dash of fun and silliness for good measure! At Hugging Face, we’re all about collaboration, creativity, and, of course, the occasional giggle while training machine learning models. 

### 🌟 What We Do

* **Models Galore**: With over **1 million models** to choose from, we’re basically the buffet of AI. You might even say we’re the “Hugging” part of “Hugging Face”—because we love to embrace all your modeling needs! 
* **Datasets aplenty**: Need data? We’re like a kid in a candy store with **250k+ datasets**. Just picture us doing the happy dance while sifting through them.
* **Spaces**: Not all heroes wear capes—some of them run applications on **12,000+ spaces**. Talk about multitasking!

### 🤝 Our Culture

At Hugging Face, we have a culture of openness. We firmly believe that if AI can be cuddly, why not our company? A typical day involves brainstorming, high-fives over virtual meetings, and perhaps a few friendly debates over whether robots can excel at karaoke. 

We also love community! With **more than 50,000 organizations** using our platform, we’re basically the ultimate matchmaking service for AI and businesses. It's like Tinder but for machine learning!

### 🎤 Who’s on Our Team?

We’re a diverse bunch of AI wizards, data geeks, and code whisperers. Think of us as a league of extraordinary nerds. With **210+ team members**, we're always looking for more talent to join our superhero squad! Whether you are an aspiring AI scientist or the next great Python poet, there's a place for you here!

### 😄 Why Customers Love Us

From household names like **Google** and **Microsoft** to upcoming enterprises, our customer lineup is more impressive than a magician pulling rabbits out of hats! People love our **enterprise solutions** and the fact that we help them build AI magic while they sip their morning coffee.

### 💼 Careers at Hugging Face

Are you ready to join the fight against boring jobs? We’re always on the lookout for passionate individuals. Whether you're a data scientist on the rise or a software engineer who can code blindfolded, we want to meet you! 

- **Perks**: Remote work, competitive pay, hilarious team memes, and free hugs (virtually, of course).
- **What We Seek**: Creative thinkers, problem solvers, and those with a knack for making AI less scary.

### 🤗 Join Us!

Ready to take your place in the future? Whether you're looking to model AI, access datasets, or just join a fantastic team of quirky individuals, come help us "hug" the world of AI! 

Visit us at [Hugging Face](https://huggingface.co). 

**Remember: Big hugs to all our models—both AI and human!** 🤖❤️

--- 

We promise we won't make you read through 100 pages of boring corporate speak. Just a few chuckles and an open invitation to change the world of AI—one hug at a time! Let's embrace the future!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>